# Обучение моделей с помощью Экспериментов Azure ML

## 1. Соединение со своим Workspace

Импорт модулей:

In [1]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core.datastore import Datastore
from azureml.train.sklearn import SKLearn

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.12.0


Создаем соединение со своей Рабоечей областью:

In [2]:
ws = Workspace.from_config()
print('Connected Workspace:', ws.name, ws.resource_group, ws.location, sep = '\n')

Connected Workspace:
ai-in-cloud-workspace
ai-in-cloud-workshop-rg
westeurope


## 2. Подготовка данных для Эксперимент

Зададим имя для эксперимента:

In [3]:
experiment_name = 'diabetes-train-model-experiment'
experiment_dir = experiment_name

Воспользуемся данными загружеными на [прошлом лабораторном занятии](../module-3/azure-datastore.ipynb) и загрузим их в папку эксперимента:

In [4]:
# Get uploaded data 
ds = ws.get_default_datastore()
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))

data = diabetes_ds.to_pandas_dataframe()
data.to_csv(f'{experiment_name}/data.csv', index=False, header=True)

Просмотрим содержимое директории эксперимента:

In [5]:
!ls diabetes-train-model-experiment

data.csv  diabetes-train-model.py


## Создадим и запускаем Эксперимент

In [ ]:
# Create an experiment
experiment = Experiment(workspace=ws, name=experiment_name)

# Create an SKLearn estimator
estimator = SKLearn(source_directory=experiment_dir,
                    entry_script='diabetes-train-model.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local'
                    )


# Run the experiment and wait result
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

See training details:

In [7]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Просмотрим метрики и выходные данные:

In [8]:
metrics = run.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.7788888888888889
AUC 0.846851712258014


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/9_azureml.log
outputs/diabetes_model.pkl


## Регистрируем обученную модель

Копируем путь до обученной модели, чтобы зарегистрировать ее.

In [9]:
model_path = run.get_file_names()[3]
model_path

'outputs/diabetes_model.pkl'

In [ ]:
run.register_model(model_path=model_path, model_name='diabetes_model',
                   tags={'Dataset':'Diabetes'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Просмотрим список зарегистрированных моделей:

In [11]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
        
    print('\n')

diabetes_model v4
	 Dataset : Diabetes
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model v3
	 Dataset : Diabetes
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model v2
	 Dataset : Diabetes
	 AUC : 0.8468519356081545
	 Accuracy : 0.7788888888888889


diabetes_model v1
	 Training context : Estimator
	 AUC : 0.8468519356081545
	 Accuracy : 0.7788888888888889


amlstudio-covid19-service v1
	 CreatedByAMLStudio : true


amlstudio-covid19-service-pipe v1
	 CreatedByAMLStudio : true


amlstudio-covid19-spread-servi v1
	 CreatedByAMLStudio : true


amlstudio-pima-diabets-service v2
	 CreatedByAMLStudio : true


amlstudio-letter-recognition-s v1
	 CreatedByAMLStudio : true


amlstudio-pima-diabetes-model v1
	 CreatedByAMLStudio : true




Проcмотрите зарегистрированную модель на портале Azure ML в разделе: `<your_workspace> > Models > diabetes_model`.